In [1]:
import pyspark
from pyspark.sql import SparkSession

# create the SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
# get all green data
df_green = spark.read.parquet('data/parquet/green/*/*')

In [6]:
# tell Spark that our DataFrame is a table
# df_trips_data.registerTempTable('trips_data')  # deprecated
df_green.createOrReplaceTempView('green')

In [7]:
# sum the total amount of fare and number of trips 
#   by revenue zone by hour from January 2020
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-06 07:00:00| 241|            169.16|             6|
|2020-01-19 19:00:00|  41| 457.2400000000003|            43|
|2020-01-23 23:00:00| 129| 304.1300000000001|            22|
|2020-01-28 15:00:00| 129|            208.23|             9|
|2020-01-02 08:00:00|  74|1187.4199999999987|            96|
|2020-01-06 16:00:00| 129|250.42000000000002|            15|
|2020-01-07 10:00:00| 171|             38.98|             2|
|2020-01-09 15:00:00| 260| 242.6800000000001|            19|
|2020-01-16 20:00:00|  82| 424.1900000000001|            27|
|2020-01-21 16:00:00|  33| 917.3599999999994|            38|
|2020-01-08 17:00:00| 140|            313.81|             8|
|2020-01-22 08:00:00| 244| 285.3999999999999|            14|
|2020-01-30 14:00:00|  74| 738.6799999999996|            49|
|2020-01-03 07:00:00|  7

In [9]:
# save these results to a report in 20 partitions
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [13]:
# do the same as above for yellow data
df_yellow = spark.read.parquet('data/parquet/yellow/*/*')

df_yellow.createOrReplaceTempView('yellow')

df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')